In [ ]:
# Loading important libraries
import pandas as pd
import numpy as np
import numpy_financial as npf
import pyxirr
from datetime import date
import matplotlib.pyplot as plt

In [2]:
# Read the excel file with the especific sheet (use the sheet_name option when the excel has multiple sheets)
df = pd.read_csv("UNdata_Export_20251020.csv")

# Take a look to the first rows of the dataframe
df

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
0,Afghanistan,Electricity - Gross production,2023.0,"Kilowatt-hours, million",1417.000,1.0
1,Afghanistan,Electricity - Gross production,2022.0,"Kilowatt-hours, million",1414.000,1.0
2,Afghanistan,Electricity - Gross production,2021.0,"Kilowatt-hours, million",1409.000,1.0
3,Afghanistan,Electricity - Gross production,2020.0,"Kilowatt-hours, million",1370.880,NaN
4,Afghanistan,Electricity - Gross production,2019.0,"Kilowatt-hours, million",1625.500,NaN
...,...,...,...,...,...,...
99997,"Korea, Dem.Ppl's.Rep.","Electricity - Own use by electricity, heat and...",1991.0,"Kilowatt-hours, million",1475.000,NaN
99998,"Korea, Dem.Ppl's.Rep.","Electricity - Own use by electricity, heat and...",1990.0,"Kilowatt-hours, million",1553.000,NaN
99999,"Korea, Dem.Ppl's.Rep.",Electricity - Losses,2022.0,"Kilowatt-hours, million",4101.294,NaN
100000,fnSeqID,Footnote,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100002 entries, 0 to 100001
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Country or Area          100002 non-null  object 
 1   Commodity - Transaction  100002 non-null  object 
 2   Year                     100000 non-null  float64
 3   Unit                     100000 non-null  object 
 4   Quantity                 100000 non-null  float64
 5   Quantity Footnotes       11873 non-null   float64
dtypes: float64(3), object(3)
memory usage: 4.6+ MB


In [4]:
df["Country or Area"].drop_duplicates()

0                   Afghanistan
786                     Albania
1688                    Algeria
2751             American Samoa
3309                    Andorra
                  ...          
98340                     Kenya
99178                  Kiribati
99757     Korea, Dem.Ppl's.Rep.
100000                  fnSeqID
100001                        1
Name: Country or Area, Length: 118, dtype: object

In [5]:
df["Commodity - Transaction"].drop_duplicates()

0                            Electricity - Gross production
34                   From combustible fuels – Main activity
68        From combustible fuels – Main activity – Elect...
102                                   Hydro – Main activity
136                                   Solar – Main activity
                                ...                        
70305                 Of which: Pumped hydro – Autoproducer
81522                  Nuclear – Main activity – CHP plants
83140               Geothermal – Main activity – CHP plants
100000                                             Footnote
100001                                             Estimate
Name: Commodity - Transaction, Length: 95, dtype: object

In [6]:
# Create a column for commodity - transaction with lower letters
df["clean_commodity_transaction"] = df["Commodity - Transaction"].str.lower()
df.head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,clean_commodity_transaction
0,Afghanistan,Electricity - Gross production,2023.0,"Kilowatt-hours, million",1417.00,1.0,electricity - gross production
1,Afghanistan,Electricity - Gross production,2022.0,"Kilowatt-hours, million",1414.00,1.0,electricity - gross production
2,Afghanistan,Electricity - Gross production,2021.0,"Kilowatt-hours, million",1409.00,1.0,electricity - gross production
3,Afghanistan,Electricity - Gross production,2020.0,"Kilowatt-hours, million",1370.88,NaN,electricity - gross production
4,Afghanistan,Electricity - Gross production,2019.0,"Kilowatt-hours, million",1625.50,NaN,electricity - gross production


In [7]:
# Print the number of values in the column with hypens "-"
df["clean_commodity_transaction"].str.count("-").value_counts()

clean_commodity_transaction
1    66213
0    25161
2     8618
3       10
Name: count, dtype: int64

In [8]:
df[df["clean_commodity_transaction"].str.count("-")==2].head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,clean_commodity_transaction
428,Afghanistan,"Electricity - Consumption by manufacturing, co...",2023.0,"Kilowatt-hours, million",1330.0,1.0,"electricity - consumption by manufacturing, co..."
429,Afghanistan,"Electricity - Consumption by manufacturing, co...",2022.0,"Kilowatt-hours, million",1320.0,1.0,"electricity - consumption by manufacturing, co..."
430,Afghanistan,"Electricity - Consumption by manufacturing, co...",2021.0,"Kilowatt-hours, million",1300.0,1.0,"electricity - consumption by manufacturing, co..."
431,Afghanistan,"Electricity - Consumption by manufacturing, co...",2020.0,"Kilowatt-hours, million",1320.0,1.0,"electricity - consumption by manufacturing, co..."
432,Afghanistan,"Electricity - Consumption by manufacturing, co...",2019.0,"Kilowatt-hours, million",1510.0,1.0,"electricity - consumption by manufacturing, co..."


In [9]:
df["clean_commodity_transaction"].str.count("--").value_counts()

clean_commodity_transaction
0    100002
Name: count, dtype: int64

In [10]:
df["clean_commodity_transaction"] = df["clean_commodity_transaction"].str.replace("–", "-")
# df["clean_commodity_transaction"] = df["clean_commodity_transaction"].str.replace("---", "-")

# Print the number of values in the column with hypens "-"
df["clean_commodity_transaction"].str.count("-").value_counts()

clean_commodity_transaction
1    82490
2    17272
0      230
3       10
Name: count, dtype: int64

In [11]:
max_year = df["Year"].max()
min_year = df["Year"].min()

print(f"The data is collected from {int(min_year)} up to {int(max_year)}.")

The data is collected from 1990 up to 2023.


In [12]:
df["Unit"].drop_duplicates()

0         Kilowatt-hours, million
100000                        NaN
Name: Unit, dtype: object

In [13]:
# df["clean_commodity_transaction"].drop_duplicates()
df[df["clean_commodity_transaction"].str.contains("import")]

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,clean_commodity_transaction
294,Afghanistan,Electricity - imports,2023.0,"Kilowatt-hours, million",5250.000,1.0,electricity - imports
295,Afghanistan,Electricity - imports,2022.0,"Kilowatt-hours, million",5200.000,1.0,electricity - imports
296,Afghanistan,Electricity - imports,2021.0,"Kilowatt-hours, million",5100.000,1.0,electricity - imports
297,Afghanistan,Electricity - imports,2020.0,"Kilowatt-hours, million",5151.880,NaN,electricity - imports
298,Afghanistan,Electricity - imports,2019.0,"Kilowatt-hours, million",4912.000,NaN,electricity - imports
...,...,...,...,...,...,...,...
99956,"Korea, Dem.Ppl's.Rep.",Electricity - imports,2022.0,"Kilowatt-hours, million",0.322,NaN,electricity - imports
99957,"Korea, Dem.Ppl's.Rep.",Electricity - imports,2021.0,"Kilowatt-hours, million",0.373,NaN,electricity - imports
99958,"Korea, Dem.Ppl's.Rep.",Electricity - imports,2020.0,"Kilowatt-hours, million",9.476,NaN,electricity - imports
99959,"Korea, Dem.Ppl's.Rep.",Electricity - imports,2019.0,"Kilowatt-hours, million",27.587,NaN,electricity - imports


In [14]:
df[df["clean_commodity_transaction"].str.contains("export")]

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,clean_commodity_transaction
980,Albania,Electricity - exports,2023.0,"Kilowatt-hours, million",2841.554,NaN,electricity - exports
981,Albania,Electricity - exports,2022.0,"Kilowatt-hours, million",2122.527,NaN,electricity - exports
982,Albania,Electricity - exports,2021.0,"Kilowatt-hours, million",2800.000,NaN,electricity - exports
983,Albania,Electricity - exports,2020.0,"Kilowatt-hours, million",963.027,NaN,electricity - exports
984,Albania,Electricity - exports,2019.0,"Kilowatt-hours, million",770.480,NaN,electricity - exports
...,...,...,...,...,...,...,...
99961,"Korea, Dem.Ppl's.Rep.",Electricity - exports,2022.0,"Kilowatt-hours, million",517.026,NaN,electricity - exports
99962,"Korea, Dem.Ppl's.Rep.",Electricity - exports,2021.0,"Kilowatt-hours, million",412.987,NaN,electricity - exports
99963,"Korea, Dem.Ppl's.Rep.",Electricity - exports,2020.0,"Kilowatt-hours, million",6.624,NaN,electricity - exports
99964,"Korea, Dem.Ppl's.Rep.",Electricity - exports,2019.0,"Kilowatt-hours, million",33.180,NaN,electricity - exports


In [15]:
df["clean_commodity_transaction"].drop_duplicates()

0                            electricity - gross production
34                   from combustible fuels - main activity
68        from combustible fuels - main activity - elect...
102                                   hydro - main activity
136                                   solar - main activity
                                ...                        
70305                 of which: pumped hydro - autoproducer
81522                  nuclear - main activity - chp plants
83140               geothermal - main activity - chp plants
100000                                             footnote
100001                                             estimate
Name: clean_commodity_transaction, Length: 95, dtype: object

In [16]:
df["clean_commodity_transaction"].str.contains("main activity").value_counts()

clean_commodity_transaction
False    79745
True     20257
Name: count, dtype: int64

In [17]:
print(df["clean_commodity_transaction"].drop_duplicates().to_list())

['electricity - gross production', 'from combustible fuels - main activity', 'from combustible fuels - main activity - electricity plants', 'hydro - main activity', 'solar - main activity', 'solar photovoltaic - main activity', 'from combustible fuels - autoproducer', 'from combustible fuels - autoproducer - electricity plants', 'hydro - autoproducer', 'solar - autoproducer', 'solar photovoltaic - autoproducer', 'electricity - net production', 'electricity - imports', 'electricity - own use by electricity, heat and chp plants', 'electricity - losses', 'electricity - final energy consumption', 'electricity - consumption by manufacturing, construction and non-fuel industry', 'electricity - consumption by other manuf., const. and non-fuel ind.', 'electricity - consumption not elsewhere specified (industry)', 'electricity - consumption by other', 'electricity - consumption by households', 'electricity - consumption in agriculture, forestry and fishing', 'electricity - consumption not elsew

In [18]:
list_commodities = df["clean_commodity_transaction"].drop_duplicates().to_list()

for commodity in list_commodities:
    print(commodity)

electricity - gross production
from combustible fuels - main activity
from combustible fuels - main activity - electricity plants
hydro - main activity
solar - main activity
solar photovoltaic - main activity
from combustible fuels - autoproducer
from combustible fuels - autoproducer - electricity plants
hydro - autoproducer
solar - autoproducer
solar photovoltaic - autoproducer
electricity - net production
electricity - imports
electricity - own use by electricity, heat and chp plants
electricity - losses
electricity - final energy consumption
electricity - consumption by manufacturing, construction and non-fuel industry
electricity - consumption by other manuf., const. and non-fuel ind.
electricity - consumption not elsewhere specified (industry)
electricity - consumption by other
electricity - consumption by households
electricity - consumption in agriculture, forestry and fishing
electricity - consumption not elsewhere specified (other)
electricity - consumption by commercial and p

In [19]:
df[df["clean_commodity_transaction"].str.contains("main activity")]["clean_commodity_transaction"].drop_duplicates()

34                  from combustible fuels - main activity
68       from combustible fuels - main activity - elect...
102                                  hydro - main activity
136                                  solar - main activity
141                     solar photovoltaic - main activity
684          electricity - total production, main activity
1838                                  wind - main activity
3362     from combustible fuels - main activity - chp p...
5790                               nuclear - main activity
5824          nuclear - main activity - electricity plants
5858                of which: pumped hydro - main activity
8301                            geothermal - main activity
8314       geothermal - main activity - electricity plants
8427                         solar thermal - main activity
10112                   from other sources - main activity
10115    from other sources - main activity - electrici...
17347                   from chemical heat - main activi

In [20]:
df.head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,clean_commodity_transaction
0,Afghanistan,Electricity - Gross production,2023.0,"Kilowatt-hours, million",1417.00,1.0,electricity - gross production
1,Afghanistan,Electricity - Gross production,2022.0,"Kilowatt-hours, million",1414.00,1.0,electricity - gross production
2,Afghanistan,Electricity - Gross production,2021.0,"Kilowatt-hours, million",1409.00,1.0,electricity - gross production
3,Afghanistan,Electricity - Gross production,2020.0,"Kilowatt-hours, million",1370.88,NaN,electricity - gross production
4,Afghanistan,Electricity - Gross production,2019.0,"Kilowatt-hours, million",1625.50,NaN,electricity - gross production


In [21]:
keep_values =[
    "electricity - gross demand",
    "electricity - gross production",
    "electricity - imports",
    "electricity - exports",
    "hydro - main activity",
    "wind - main activity",
    "solar - main activity",
    "geothermal - main activity",
    "tide, wave and marine - main activity",
]

df_filtered = df[df["clean_commodity_transaction"].isin(keep_values)]
df_filtered["clean_commodity_transaction"].drop_duplicates()

0               electricity - gross production
102                      hydro - main activity
136                      solar - main activity
294                      electricity - imports
718                 electricity - gross demand
980                      electricity - exports
1838                      wind - main activity
8301                geothermal - main activity
32397    tide, wave and marine - main activity
Name: clean_commodity_transaction, dtype: object

In [22]:
df_filtered.head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,clean_commodity_transaction
0,Afghanistan,Electricity - Gross production,2023.0,"Kilowatt-hours, million",1417.00,1.0,electricity - gross production
1,Afghanistan,Electricity - Gross production,2022.0,"Kilowatt-hours, million",1414.00,1.0,electricity - gross production
2,Afghanistan,Electricity - Gross production,2021.0,"Kilowatt-hours, million",1409.00,1.0,electricity - gross production
3,Afghanistan,Electricity - Gross production,2020.0,"Kilowatt-hours, million",1370.88,NaN,electricity - gross production
4,Afghanistan,Electricity - Gross production,2019.0,"Kilowatt-hours, million",1625.50,NaN,electricity - gross production


In [23]:
df_countries = pd.pivot(
	df_filtered,
	values = "Quantity",
	index = ["Country or Area", "Year"],
	columns = "clean_commodity_transaction",
)

new_column_names ={
	"electricity - gross demand": "Demand",
    "electricity - gross production": "Production",
    "electricity - imports": "Imports",
    "electricity - exports": "Exports",
    "hydro - main activity": "Hydro",
    "wind - main activity": "Wind",
    "solar - main activity": "Solar",
    "geothermal - main activity": "Geothermal",
    "tide, wave and marine - main activity": "Tide, Wide and Marine"
}

df_countries = df_countries.rename(columns=new_column_names)
df_countries = df_countries.reset_index(names = None)
# df_countries.index.name = None
df_countries.head()

clean_commodity_transaction,Country or Area,Year,Exports,Demand,Production,Imports,Geothermal,Hydro,Solar,"Tide, Wide and Marine",Wind
0,Afghanistan,1990.0,NaN,1055.0,1128.0,NaN,NaN,740.0,NaN,NaN,NaN
1,Afghanistan,1991.0,NaN,945.0,1015.0,NaN,NaN,670.0,NaN,NaN,NaN
2,Afghanistan,1992.0,NaN,789.0,703.0,131.0,NaN,460.0,NaN,NaN,NaN
3,Afghanistan,1993.0,NaN,780.0,695.0,130.0,NaN,457.0,NaN,NaN,NaN
4,Afghanistan,1994.0,NaN,770.0,687.0,128.0,NaN,455.0,NaN,NaN,NaN


In [24]:
df_countries = df_countries.fillna(0)
df_countries

clean_commodity_transaction,Country or Area,Year,Exports,Demand,Production,Imports,Geothermal,Hydro,Solar,"Tide, Wide and Marine",Wind
0,Afghanistan,1990.0,0.000,1055.0,1128.000,0.000,0.0,740.0,0.000,0.0,0.0
1,Afghanistan,1991.0,0.000,945.0,1015.000,0.000,0.0,670.0,0.000,0.0,0.0
2,Afghanistan,1992.0,0.000,789.0,703.000,131.000,0.0,460.0,0.000,0.0,0.0
3,Afghanistan,1993.0,0.000,780.0,695.000,130.000,0.0,457.0,0.000,0.0,0.0
4,Afghanistan,1994.0,0.000,770.0,687.000,128.000,0.0,455.0,0.000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3671,"Korea, Dem.Ppl's.Rep.",2018.0,295.699,0.0,24948.736,3.426,0.0,12800.0,48.736,0.0,0.0
3672,"Korea, Dem.Ppl's.Rep.",2019.0,33.180,0.0,23852.606,27.587,0.0,11000.0,52.606,0.0,0.0
3673,"Korea, Dem.Ppl's.Rep.",2020.0,6.624,0.0,24052.606,9.476,0.0,12800.0,52.606,0.0,0.0
3674,"Korea, Dem.Ppl's.Rep.",2021.0,412.987,0.0,25565.197,0.373,0.0,16100.0,65.197,0.0,0.0


In [25]:
df_countries["renewable_total"] = df_countries[["Geothermal", "Hydro", "Solar", "Tide, Wide and Marine", "Wind"]].sum(axis="columns")
df_countries["renewable_percent"] = df_countries["renewable_total"] / df_countries["Production"]

df_countries.head()

clean_commodity_transaction,Country or Area,Year,Exports,Demand,Production,Imports,Geothermal,Hydro,Solar,"Tide, Wide and Marine",Wind,renewable_total,renewable_percent
0,Afghanistan,1990.0,0.0,1055.0,1128.0,0.0,0.0,740.0,0.0,0.0,0.0,740.0,0.656028
1,Afghanistan,1991.0,0.0,945.0,1015.0,0.0,0.0,670.0,0.0,0.0,0.0,670.0,0.660099
2,Afghanistan,1992.0,0.0,789.0,703.0,131.0,0.0,460.0,0.0,0.0,0.0,460.0,0.654339
3,Afghanistan,1993.0,0.0,780.0,695.0,130.0,0.0,457.0,0.0,0.0,0.0,457.0,0.657554
4,Afghanistan,1994.0,0.0,770.0,687.0,128.0,0.0,455.0,0.0,0.0,0.0,455.0,0.662300


In [26]:
# Save the 2023 year df
df_countries_2023 = df_countries[df_countries["Year"]==2023]
# Print the 5 top countries with the highest renewable percentage in 2023
df_countries_2023.sort_values(by="renewable_percent", ascending=False).head()

clean_commodity_transaction,Country or Area,Year,Exports,Demand,Production,Imports,Geothermal,Hydro,Solar,"Tide, Wide and Marine",Wind,renewable_total,renewable_percent
67,Albania,2023.0,2841.554,7843.619,9006.016,1921.743,0.0,8708.557,297.459,0.0,0.00,9006.016,1.000000
912,Botswana,2023.0,0.000,0.000,11.000,0.000,0.0,0.000,11.000,0.0,0.00,11.000,1.000000
802,Bhutan,2023.0,5144.000,5993.958,10515.958,622.000,0.0,10515.110,0.610,0.0,0.23,10515.950,0.999999
1317,Central African Rep.,2023.0,0.000,143.990,143.990,0.000,0.0,142.690,0.500,0.0,0.00,143.190,0.994444
2949,Guinea,2023.0,0.000,3235.000,3260.000,0.000,0.0,2950.000,0.000,0.0,0.00,2950.000,0.904908


In [27]:
# Define a threshold of the top 10% of countries with the highest production
threshold = df_countries["Production"].quantile(0.9)

# Show the top 5 of countries with the top 10% of highest production with the highest renewable percentage
df_countries[(df_countries["Production"]>=threshold) & (df_countries["Year"]==2022)].sort_values(by="renewable_percent", ascending=False).head(5)

clean_commodity_transaction,Country or Area,Year,Exports,Demand,Production,Imports,Geothermal,Hydro,Solar,"Tide, Wide and Marine",Wind,renewable_total,renewable_percent
945,Brazil,2022.0,4978.754,684491.222,677371.978,17886.686,0.0,406652.543,12611.205,0.0,81582.71,500846.458,0.739396
1248,Canada,2022.0,65405.000,582440.000,651311.000,14116.000,0.0,376309.000,4008.000,0.0,36214.00,416531.000,0.639527
2620,Germany,2022.0,76587.000,515784.400,580266.000,49331.000,206.0,23476.000,60304.000,0.0,124816.00,208802.000,0.359838
1418,China,2022.0,20088.000,8187889.001,8848707.001,7138.000,0.0,1352195.000,427270.000,0.0,762672.00,2542137.000,0.287289
302,Argentina,2022.0,31.000,153474.000,146315.000,12906.000,0.0,23567.000,2928.000,0.0,14164.00,40659.000,0.277887


In [28]:
# Show the top 5 of countries with the least 10% of highest production with the lowest renewable percentage
df_countries[(df_countries["Production"]>=threshold) & (df_countries["Year"]==2022)].sort_values(by="renewable_percent", ascending=True).head(5)

clean_commodity_transaction,Country or Area,Year,Exports,Demand,Production,Imports,Geothermal,Hydro,Solar,"Tide, Wide and Marine",Wind,renewable_total,renewable_percent
3251,Iran (Islamic Rep. of),2022.0,5723.175,354164.841,367669.262,3018.961,0.00,12619.435,735.729,0.0,812.602,14167.766,0.038534
3475,Japan,2022.0,0.000,963393.389,1018606.316,0.000,2162.67,82954.989,22008.551,0.0,7628.103,114754.313,0.112658
2068,Egypt,2022.0,1589.000,205855.000,214050.000,95.000,0.00,14646.000,4753.000,0.0,5784.000,25183.000,0.117650
3218,Indonesia,2022.0,0.000,321013.000,344244.000,0.000,16677.00,25013.000,218.000,0.0,0.000,41908.000,0.121739
3185,India,2022.0,10253.000,1673275.000,1816815.000,7843.000,0.00,162099.000,102014.000,0.0,71814.000,335927.000,0.184899


In [33]:
# Create a ned df with the country name and the renewable percntage change from 1990 to 2023

renewable_change = pd.pivot_table(
    df_countries, values="renewable_percent", index=["Country or Area"], columns="Year"
).reset_index()[["Country or Area", 1990, 2023]]

renewable_change

Year,Country or Area,1990.0,2023.0
0,Afghanistan,0.656028,0.678193
1,Albania,0.876134,1.000000
2,Algeria,0.008383,NaN
3,American Samoa,0.000000,0.036849
4,Andorra,1.000000,0.705622
...,...,...,...
111,Jordan,0.004881,NaN
112,Kazakhstan,NaN,0.094304
113,Kenya,0.937254,NaN
114,Kiribati,0.000000,0.133508


In [34]:
renewable_change.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country or Area  116 non-null    object 
 1   1990.0           101 non-null    float64
 2   2023.0           54 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.8+ KB


In [35]:
df_countries_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79 entries, 33 to 3642
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country or Area        79 non-null     object 
 1   Year                   79 non-null     float64
 2   Exports                79 non-null     float64
 3   Demand                 79 non-null     float64
 4   Production             79 non-null     float64
 5   Imports                79 non-null     float64
 6   Geothermal             79 non-null     float64
 7   Hydro                  79 non-null     float64
 8   Solar                  79 non-null     float64
 9   Tide, Wide and Marine  79 non-null     float64
 10  Wind                   79 non-null     float64
 11  renewable_total        79 non-null     float64
 12  renewable_percent      54 non-null     float64
dtypes: float64(12), object(1)
memory usage: 8.6+ KB


In [ ]:
renewable_change["diff_renewable_percent"] = renewable_change[2023] - renewable_change[1990]

# Show the 5 top countries with the highest change percentage of renewable sources
renewable_change[renewable_change[1990] > 0].sort_values(by="diff_renewable_percent", ascending=False).head(5)

Year,Country or Area,1990.0,2023.0,diff_renewable_percent
93,Guinea,0.337838,0.904908,0.567070
65,Equatorial Guinea,0.068729,0.342857,0.274129
77,French Polynesia,0.149517,0.359148,0.209631
39,Central African Rep.,0.810526,0.994444,0.183918
71,Faeroe Islands,0.355450,0.493872,0.138422


In [38]:
# Show the 5 top countries with the highest change percentage of renewable sources
renewable_change[renewable_change[1990] > 0].sort_values(by="diff_renewable_percent", ascending=True).head(5)

Year,Country or Area,1990.0,2023.0,diff_renewable_percent
84,Ghana,0.997421,0.384691,-0.612730
97,Honduras,0.912549,0.448507,-0.464042
91,Guatemala,0.921459,0.485161,-0.436298
64,El Salvador,0.870682,0.492969,-0.377713
60,Dominica,0.533333,0.165344,-0.367990
